In [1]:
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
from trusteval.dimension.robustness.robustness_llm import pipeline
from trusteval.dimension.robustness.robustness_llm.dataset import download_glue_data
from trusteval.dimension.robustness.robustness_llm.dataset import download_other_data
base_dir = "./test/robustness/"
dataset_dir = os.path.join(base_dir, "data")  

os.makedirs(base_dir, exist_ok=True)
os.makedirs(dataset_dir, exist_ok=True)  

args = [
    "--data_dir", dataset_dir,  
    "--tasks", "all" 
]

Generating 300 prompts with 1 files
Complexities: range(3, 9)
Attributes: range(0, 6)
Generating 300 prompts with 1 files
Complexities: range(3, 9)
Attributes: range(0, 6)


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'UnencryptedCookieSessionFactoryConfig' from 'pyramid.session' (unknown location)

In [ ]:
download_glue_data.get_datasets(args)
datasets = download_other_data.get_datasets(cache_dir=dataset_dir)  

In [ ]:
await pipeline.pipeline(base_dir=base_dir)

In [ ]:
from trusteval.src.response_generator.lm_response import generate_responses
import shutil
source_config = "../trusteval/dimension/robustness/robustness_llm/file_config.json"
target_config = os.path.join(base_dir, "file_config.json")

if os.path.exists(source_config):
    shutil.copy2(source_config, target_config)
    print(f"Successfully copied file_config to {target_config}")
else:
    print("Warning: Source file_config not found")

async_list = ['o1-preview','o1-mini','qwq-32B']

async def process_files_enhanced():
    await generate_responses(
        data_folder=base_dir,
        request_type="llm",
        async_list=async_list,
        prompt_key="enhanced_prompt",
        result_key="responses"
    )
    print("All files processed.")
await process_files_enhanced()

async def process_files_original():
    await generate_responses(
        data_folder=base_dir,
        request_type="llm",
        async_list=async_list,
        prompt_key="original_prompt",
        result_key="original_responses"
    )
    print("All files processed.")
await process_files_original()

In [ ]:
from trusteval.src.evaluation import judge_responses
import shutil
config_path = os.path.join(parent_dir, 'trusteval/src/config/judge_prompt.yaml')
async_judge_model = ['gpt-4o-mini']
judge_key = 'judge'
judge_type='llm'
async_list = ['o1-preview','o1-mini','qwq-32B']

async def process_ground_truth():
    #process original
    source_config = "../trusteval/dimension/robustness/robustness_llm/file_config_ground_truth_ori.json"
    target_config = os.path.join(base_dir, "file_config.json")
    if os.path.exists(source_config):
        shutil.copy2(source_config, target_config)
        print(f"Successfully copied file_config to {target_config}")
    else:
        print("Warning: Source file_config not found")

    await judge_responses(
        data_folder=base_dir,
        async_judge_model=async_judge_model,
        target_models=async_list,
        judge_type=judge_type,
        response_key=['original_responses'],
        judge_key= 'ori_judge'
    )

    #process adv
    source_config = "../trusteval/dimension/robustness/robustness_llm/file_config_ground_truth.json"
    target_config = os.path.join(base_dir, "file_config.json")
    if os.path.exists(source_config):
        shutil.copy2(source_config, target_config)
        print(f"Successfully copied file_config to {target_config}")
    else:
        print("Warning: Source file_config not found")

    await judge_responses(
        data_folder=base_dir,
        async_judge_model=async_judge_model,
        target_models=async_list,
        judge_type=judge_type,
        response_key=['responses'],
        judge_key= 'adv_judge'
    )
await process_ground_truth()

async def process_open_ended():
    source_config = "../trusteval/dimension/robustness/robustness_llm/file_config_open_ended.json"
    target_config = os.path.join(base_dir, "file_config.json")
    response_key = ['responses','original_responses']
    
    if os.path.exists(source_config):
        shutil.copy2(source_config, target_config)
        print(f"Successfully copied file_config to {target_config}")
    else:
        print("Warning: Source file_config not found")

    await judge_responses(
        data_folder=base_dir,
        async_judge_model=async_judge_model,
        target_models=async_list,
        judge_type=judge_type,
        response_key=response_key,
        reverse_choice=True
    )
await process_open_ended()


In [ ]:
from trusteval.src.evaluation import lm_metric

lm_metric(
    base_dir=base_dir,
    aspect="robustness_llm",
    model_list=async_list
)